#BehaviorScoringRun.ipynb




##🪰 This notebook is the entry point for the **BehaviorScoring** pipeline.

---

It keeps
implementation details inside modules and streamlines the workflow: set up the
environment, select paths, run the analysis, and produce consistent scored outputs.The emphasis is a user-friendly, reproducible flow: clear separation of config from logic, minimal editing in one place, and version-pinned runs.

This notebook guides you through a standard sequence: configure once, stage inputs, run scoring, and sync outputs. Edit only the cells marked **EDIT ME**; everything else should run as-is.


#### Run order
```
1) Setup scoring environment  
2) Import configs & stage to local mirrors  
3) Run pipeline  
4) Wrap-up cleaning
```




#### Dependencies
```
BehaviorScoringMain.py
BehaviorScoringFunctions.py
BehaviorScoringConfig.py
BehaviorScoringColabConfig.py
ExperimentConfig.py
PathConfig.py
```

## 1. Setup scoring environment

Prepare a clean, reproducible runtime and point the pipeline to your experiment.

- Pin core library versions and show environment details.
- Mount Google Drive and verify access.
- Load **PathConfig** in Production or TEMP (dev) mode (EDIT ME).

In [1]:
# Pin core library versions and show environment details

# python==3.11.13
# Pin libs to versions compatible with Colab's preinstalled stack.
!pip install -q \
  numpy==2.0.2 \
  pandas==2.2.2 \
  scikit-learn==1.6.1 \
  scipy==1.14.1

import sys, platform, numpy, pandas, sklearn, scipy
print("Python:", sys.version.split()[0], "| Platform:", platform.platform())
print("NumPy :", numpy.__version__,
      "| pandas :", pandas.__version__,
      "| sklearn :", sklearn.__version__,
      "| SciPy :", scipy.__version__)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 418.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 15.5 MB/s eta 0:00:00
Python: 3.11.13 | Platform: Linux-6.1.123+-x86_64-with-glibc2.35
NumPy : 2.0.2 | pandas : 2.2.2 | sklearn : 1.6.1 | SciPy : 1.14.1


In [2]:
#Mount Google Drive

from pathlib import Path
from google.colab import drive

drive.mount("/content/drive")
drive_root = Path("/content/drive/My Drive")
assert drive_root.exists(), "Drive root not found after mount."
print("Drive mounted ✓")


Mounted at /content/drive
Drive mounted ✓


In [3]:
# PathConfig: production or TEMP (dev)

# ============================ EDIT ME (choose one) ==========================
USE_TEMP_PATHCONFIG = True  # True = DEV (inject root); False = PROD (real file)

# PROD: absolute path to your real PathConfig.py (with a real __EXP_ROOT__)
pPathConfig = "/content/drive/My Drive/YourExperiment/Protocols/Codes/Config/PathConfig.py"

# DEV: PathConfig.py to inject, and the temporary experimental root to use
TEMP_PATHCONFIG      = "/content/drive/My Drive/FHAnalysisPipeline/10DB/Protocols/Codes/Config/PathConfig.py"
TEMP_EXPERIMENT_ROOT = "/content/drive/My Drive/FHAnalysisPipeline/10DB"
# ===========================================================================

import sys, types, importlib.util

if USE_TEMP_PATHCONFIG:
    p_pathconfig = Path(TEMP_PATHCONFIG)
    exp_root     = Path(TEMP_EXPERIMENT_ROOT)
    assert p_pathconfig.exists(), "TEMP PathConfig.py not found."
    assert exp_root.exists(),     "TEMP experimental root not found."

    # Inject __EXP_ROOT__ in-memory (no disk edits)
    code = p_pathconfig.read_text().replace('"__EXP_ROOT__"', f'"{exp_root.as_posix()}"')
    PathConfig = types.ModuleType("PathConfig")
    exec(compile(code, str(p_pathconfig), "exec"), PathConfig.__dict__)
    print("TEMP PathConfig injected ✓")

else:
    p_pathconfig = Path(pPathConfig)
    assert p_pathconfig.exists(), "PathConfig.py not found."
    spec = importlib.util.spec_from_file_location("PathConfig", str(p_pathconfig))
    PathConfig = importlib.util.module_from_spec(spec)
    sys.modules["PathConfig"] = PathConfig
    spec.loader.exec_module(PathConfig)
    print("Production PathConfig loaded ✓")

# Make Codes importable and preload the Config package
sys.path.insert(0, str(Path(PathConfig.pCodes)))
import importlib
importlib.import_module("Config")
sys.modules["Config.PathConfig"] = PathConfig  # satisfy 'from Config import PathConfig'

TEMP PathConfig injected ✓


## 2. Import configs & stage to local mirrors

Validate inputs on Drive, create fast local mirrors under `/content`, and stage
only what needs work. Then build a local `PathConfig` that points the pipeline
to the staged data and provides fresh convenience globs.

- Import config modules and validate Drive inputs.
- Create local mirrors and stage inputs/placeholders.
- Build `LocalPathConfig` (Paths + globs) and verify counts.

In [4]:
from Config import ExperimentConfig
from Config import BehaviorScoringConfig
from Config import BehaviorScoringColabConfig
import shutil, time

# Optional: start fresh by clearing the local workspace
CLEAN_LOCAL_ON_START = True

# 1) Load canonical Drive paths
drive_root, drive_paths = BehaviorScoringColabConfig.load_configs(PathConfig)

# 2) Optionally clean local workspace for this experiment
local_base = Path("/content/exp_runs") / drive_root.name
if CLEAN_LOCAL_ON_START and local_base.exists():
    shutil.rmtree(local_base, ignore_errors=True)

# 3) Validate required inputs (auto-detect pose if not given)
pose_scoring = BehaviorScoringColabConfig.validate_inputs(drive_paths)

# 4) Create local mirrors at /content/exp_runs/<exp>
local_paths = BehaviorScoringColabConfig.local_mirrors(drive_root, drive_paths)

# 5) Stage inputs + placeholders for existing outputs
staged = BehaviorScoringColabConfig.stage_to_local(drive_paths, local_paths, pose_scoring)

# 6) Build PathConfig rebased to /content with fresh convenience globs
LocalPathConfig = BehaviorScoringColabConfig.make_local_pathconfig(PathConfig, local_paths)

# 7) Verify globs (generators) point to local mirrors
def _len_glob(g):
    try: return sum(1 for _ in g)
    except Exception: return 0

================= LOADING FILES FROM DRIVE =================

   Estimated: ~01:55 at 2.6 MB/s for 26/26 files

   Inputs        : Tracked=13, Pose=13
   Existing outs : Scored=0, ScoredError=0
   Staging time  : 00:25

======================= READY TO RUN =======================


## 3. Run pipeline

Import the scoring modules, start a silent background sync (so outputs are
pushed to Drive in small batches), and run the main behavior-scoring loop.

- Import scoring modules used by the pipeline.
- Start background sync (triggers after a small batch).
- Run scoring, then always do a final sync.

In [5]:
from BehaviorScoring import BehaviorScoringFunctions as BSF
from BehaviorScoring.BehaviorScoringMain import behavior_scoring_main

# ============================ EDIT ME (run controls) =========================
BATCH_SYNC_SIZE = 50   # trigger sync after this many new final files
# ============================================================================

# Start background sync (silent); pushes after an exact batch of new files
BehaviorScoringColabConfig.start_background_sync(
    local_paths, drive_paths, batch_size=int(BATCH_SYNC_SIZE))

# Run the pipeline
try:
    behavior_scoring_main(LocalPathConfig, ExperimentConfig, BehaviorScoringConfig, BSF)
finally:
    # Always stop sync and push any remaining files
    BehaviorScoringColabConfig.stop_background_sync()
    BehaviorScoringColabConfig.sync_outputs_back(local_paths, drive_paths, verbose=True)

PROCESSING: /content/exp_runs/10DB
POSE:    TRUE

FILES FOUND   -----------------------------------------------------   13
TO SCORE      -----------------------------------------------------   13
SKIPPING      ------------------------------------------------------   0
              ----------------------------   scored: 0   ---   errors: 0

SCORING: file 1/13 (0.00 s/file – 00h00 eta)
SCORING: file 2/13 (28.92 s/file – 00h05 eta)
SCORING: file 3/13 (28.86 s/file – 00h05 eta)
SCORING: file 4/13 (28.86 s/file – 00h04 eta)
SCORING: file 5/13 (28.81 s/file – 00h04 eta)
SCORING: file 6/13 (28.81 s/file – 00h03 eta)
SCORING: file 7/13 (28.86 s/file – 00h03 eta)
SCORING: file 8/13 (28.82 s/file – 00h02 eta)
SCORING: file 9/13 (28.90 s/file – 00h02 eta)
SCORING: file 10/13 (28.90 s/file – 00h01 eta)
SCORING: file 11/13 (28.90 s/file – 00h01 eta)
SCORING: file 12/13 (28.95 s/file – 00h00 eta)
├────  ERROR: Insufficient exploration during baseline period. (Walk 14% | > 20% allowed)
└────  CR-Emp

## 4. Wrap-up cleaning

After you run the separate “Run pipeline” cells (in your workflow notebook),
use this section to review output locations and optionally remove local mirrors.

- Optionally remove local mirrors after final sync.
- Confirm the cleanup result.

In [ ]:
# Optional: remove local mirrors now
CLEAN_LOCAL_AT_END = True  # set True to clean up after sync
if CLEAN_LOCAL_AT_END:
    shutil.rmtree(local_paths.root, ignore_errors=True)
    print("Local mirrors removed:", local_paths.root)
else:
    print("Local mirrors preserved:", local_paths.root)
